<a href="https://colab.research.google.com/github/omkar-salunke/Trading_algos/blob/main/Strategy_backtest_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing data 
!git clone https://github.com/omkar-salunke/Trading_algos.git

fatal: destination path 'Trading_algos' already exists and is not an empty directory.


In [ ]:

# from google.colab import files
# zip=files.download("/content/Trading_algos/Data_NSE.zip")
# # Data taken from kaggle https://www.kaggle.com/rohanrao/nifty50-stock-market-data


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import zipfile
with zipfile.ZipFile("/content/Trading_algos/Data_NSE.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/Trading_algos")

In [ ]:
!pip install ta
# Install backtesting infrastructure
# https://pypi.org/project/Backtesting/
!pip install backtesting
!apt-get install nodejs -y
!npm install -g --unsafe-perm ijavascript
!ijsinstall --install=global


Reading package lists... Done
Building dependency tree       
Reading state information... Done
nodejs is already the newest version (8.10.0~dfsg-2ubuntu0.4).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
npm WARN deprecated uuid@3.4.0: Please upgrade  to version 7 or higher.  Older versions may use Math.random() in certain circumstances, which is known to be problematic.  See https://v8.dev/blog/math-random for details.
/usr/bin/ijs -> /usr/lib/node_modules/ijavascript/bin/ijavascript.js
/usr/bin/ijsconsole -> /usr/lib/node_modules/ijavascript/bin/ijsconsole.js
/usr/bin/ijsinstall -> /usr/lib/node_modules/ijavascript/bin/ijsinstall.js
/usr/bin/ijskernel -> /usr/lib/node_modules/ijavascript/lib/kernel.js
/usr/bin/ijsnotebook -> /usr/lib/node_modules/ijavascript/bin/ijsnotebook.js
+ ijavascript@5.2.0
updated 1 package in 1.609s


   ╭────────────────────────────────────────────────────────────────╮
   │                                                                │
 

In [ ]:
# Preprocess Open, High, Low, Close (OHLC) data for Nifty index from 2009 to 2020
import pandas as pd
stock_data = pd.read_csv('/content/Trading_algos/BAJAJ-AUTO.csv');
stock_data.drop(columns=['Series','Symbol','Prev Close', 'VWAP', 'Turnover', 'Trades', 'Deliverable Volume', '%Deliverble'], inplace=True)
stock_data.Date= pd.to_datetime(stock_data.Date,format='%Y-%m-%d')
stock_data.set_index('Date',drop=True, inplace=True)
stock_data.columns

Index(['Open', 'High', 'Low', 'Last', 'Close', 'Volume'], dtype='object')

In [ ]:
stock_data.head()

,Open,High,Low,Last,Close,Volume
Date,,,,,,
2008-05-26,898.00,898.0,551.35,600.25,604.75,3972485
2008-05-27,624.70,639.0,580.30,595.50,593.15,1751063
2008-05-28,561.65,621.9,561.65,605.10,608.15,1652355
2008-05-29,619.40,619.4,576.00,600.00,599.45,669269
2008-05-30,605.40,607.0,538.00,576.25,571.70,1262117


In [ ]:
# Backtest a simple EMA -> SMA crossover strategy on above data

import backtesting
backtesting.set_bokeh_output(notebook=True)
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

import pandas as pd
import ta.trend as tr


# BOKEH_RESOURCES=inline
class EmaSmaCross(Strategy):
    def init(self):
      
      cp = pd.Series(self.data.Close)
      
      self.ema1 = self.I(tr.ema_indicator,close=cp, window=20);
      self.ema2 = self.I(tr.sma_indicator,close=cp, window=50);
      self.ema3 = self.I(tr.sma_indicator,close=cp, window=200)
        

    def next(self):
      # Buy or sell on ema -> sma cross over
      # Have a 200 pts stop loss for long trades and 130 points for short trades
      if crossover(self.ema1, self.ema2):
        self.buy(sl=self.data.High-200)
      elif crossover(self.ema2, self.ema1):
        self.sell(sl=self.data.Low+130)
      elif crossover(self.ema2, self.ema3):
        self.sell(sl=self.data.Low+100)     

bt = Backtest(stock_data, EmaSmaCross, commission=.002,cash=1000000,
              exclusive_orders=True)
# Run the backtest
stats = bt.run()
 
# Print the performance metrics for this strategy 
print(stats)

# Plot the price movements, trades and other meta-data such as drawdown etc.
bt.plot(resample=True);

/usr/local/lib/python3.7/dist-packages/backtesting/_plotting.py:45: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Start                     2008-05-26 00:00:00
End                       2021-04-30 00:00:00
Duration                   4722 days 00:00:00
Exposure Time [%]                     58.2761
Equity Final [$]                       555060
Equity Peak [$]                   1.01098e+06
Return [%]                            -44.494
Buy & Hold Return [%]                  533.94
Return (Ann.) [%]                    -4.52727
Volatility (Ann.) [%]                 18.0734
Sharpe Ratio                                0
Sortino Ratio                               0
Calmar Ratio                                0
Max. Drawdown [%]                     -62.759
Avg. Drawdown [%]                    -24.0245
Max. Drawdown Duration     4400 days 00:00:00
Avg. Drawdown Duration     1472 days 00:00:00
# Trades                                   74
Win Rate [%]                          29.7297
Best Trade [%]                        21.1841
Worst Trade [%]                      -15.7654
Avg. Trade [%]                    

**Hammer Candle backtesting**

In [ ]:
df= pd.read_csv('/content/Trading_algos/BAJAJ-AUTO.csv', index_col='Date')
df.index = pd.to_datetime(df.index)
df.tail()

,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
Date,,,,,,,,,,,,,,
2021-04-26,BAJAJ-AUTO,EQ,3672.50,3682.20,3749.00,3676.0,3739.00,3738.85,3719.79,480016,1.785560e+14,36835.0,155563,0.3241
2021-04-27,BAJAJ-AUTO,EQ,3738.85,3738.85,3798.00,3727.2,3795.00,3785.50,3764.89,518487,1.952048e+14,47765.0,140987,0.2719
2021-04-28,BAJAJ-AUTO,EQ,3785.50,3810.05,3954.00,3807.0,3887.15,3889.75,3909.99,1252958,4.899048e+14,78857.0,190766,0.1523
2021-04-29,BAJAJ-AUTO,EQ,3889.75,3943.00,3954.30,3807.1,3818.95,3836.45,3880.32,1335444,5.181950e+14,92106.0,168500,0.1262
2021-04-30,BAJAJ-AUTO,EQ,3836.45,3805.00,3966.35,3800.0,3827.00,3833.75,3869.17,1340273,5.185744e+14,79430.0,229768,0.1714


In [ ]:
# only considering the recent data after corona crash
recent = df[df.index > '2021-03-30']

In [ ]:
df.drop(['Symbol','Series','Prev Close','Trades','Deliverable Volume','%Deliverble','Last','VWAP','Volume','Turnover'],inplace=True,axis=1)

In [ ]:
import backtrader as bt
# Most of the code directly copied from the backtrader website********************************************
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        
        self.dataclose = self.datas[0].close
        self.datahigh = self.datas[0].High
        self.dataopen = self.datas[0].Open
        self.datalow = self.datas[0].Low
        # To keep track of pending orders
        self.order = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

       
        if not self.position:
            if (((self.datahigh - self.datalow) > 3 * (self.dataopen - self.dataclose)) and((self.dataclose - self.datalow) / (.001 + self.datahigh - self.datalow) > 0.6) and ((self.dataopen - self.datalow) / (.001 + self.datahigh - self.datalow) > 0.6)):
                self.order = self.buy() 


        else:

            # Randomly selling after 3 timecyles...
            if len(self) >= (self.bar_executed + 3):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

ModuleNotFoundError: ignored

In [ ]:
import backtrader as bt
cerebro = bt.Cerebro()
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
data = bt.feeds.PandasData(dataname=recent)
cerebro.adddata(data)
cerebro.addstrategy(TestStrategy)
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())


ModuleNotFoundError: ignored

**Running the same algorithm on all the stocks**

In [ ]:
!pip install mplfinance
!pip install backtrader
import glob
import pandas as pd
import backtrader as bt

     |████████████████████████████████| 71kB 3.8MB/s 
     |████████████████████████████████| 419kB 5.1MB/s 


In [ ]:
class Hammer_candle(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        
        self.dataclose = self.datas[0].close
        self.datahigh = self.datas[0].High
        self.dataopen = self.datas[0].Open
        self.datalow = self.datas[0].Low
        # To keep track of pending orders
        self.order = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

       
        if not self.position:
            if (((self.datahigh - self.datalow) > 3 * (self.dataopen - self.dataclose)) and((self.dataclose - self.datalow) / (.001 + self.datahigh - self.datalow) > 0.6) and ((self.dataopen - self.datalow) / (.001 + self.datahigh - self.datalow) > 0.6)):
                self.order = self.buy() 


        else:

            # Randomly selling after 3 timecyles...
            if len(self) >= (self.bar_executed + 3):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [ ]:

row=0
Summary_df=pd.DataFrame()
glued_data = pd.DataFrame()
for file_name in glob.glob('/content/Trading_algos/'+'*.csv'):
  if 'cerebro' in globals():
    del cerebro
  

  try:
    cerebro = bt.Cerebro()
    df= pd.read_csv(file_name, index_col='Date')
    df.index = pd.to_datetime(df.index) 
    df.drop(['Symbol','Series','Prev Close','Trades','Deliverable Volume','%Deliverble','Last','VWAP','Volume','Turnover'],inplace=True,axis=1)
    # For the period after specific date
    recent = df[df.index > '2020-05-29']
    # print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    intial_val=cerebro.broker.getvalue()
    
    data = bt.feeds.PandasData(dataname=recent)
    cerebro.adddata(data)
    cerebro.addstrategy(Hammer_candle)
    cerebro.run()
    final_val=cerebro.broker.getvalue()
    # print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    Summary_df.loc[row, 'stock_name'] = file_name.split("/")[-1]
    Summary_df.loc[row, 'intial_portfolio_val'] = intial_val
    Summary_df.loc[row, 'final_portfolio_val'] = final_val
    row+=1
  except Exception:
    pass

Streaming output truncated to the last 5000 lines.
2020-11-14, SELL CREATE, 514.00
2020-11-17, SELL EXECUTED, 517.00
2020-11-17, Close, 518.40
2020-11-18, BUY EXECUTED, 514.00
2020-11-18, Close, 513.00
2020-11-19, Close, 507.35
2020-11-20, Close, 503.25
2020-11-23, Close, 512.85
2020-11-23, SELL CREATE, 512.85
2020-11-24, SELL EXECUTED, 513.50
2020-11-24, Close, 520.15
2020-11-25, BUY EXECUTED, 521.20
2020-11-25, Close, 506.20
2020-11-26, Close, 511.70
2020-11-27, Close, 511.65
2020-12-01, Close, 539.35
2020-12-01, SELL CREATE, 539.35
2020-12-02, SELL EXECUTED, 540.90
2020-12-02, Close, 544.65
2020-12-03, BUY EXECUTED, 550.00
2020-12-03, Close, 548.25
2020-12-04, Close, 569.05
2020-12-07, Close, 582.30
2020-12-08, Close, 568.70
2020-12-08, SELL CREATE, 568.70
2020-12-09, SELL EXECUTED, 571.00
2020-12-09, Close, 569.55
2020-12-10, BUY EXECUTED, 570.00
2020-12-10, Close, 565.20
2020-12-11, Close, 568.10
2020-12-14, Close, 576.25
2020-12-15, Close, 571.65
2020-12-15, SELL CREATE, 571.65
2

,stock_name,intial_portfolio_val,final_portfolio_val
0,HINDALCO.csv,10000.0,10207.35
1,RELIANCE.csv,10000.0,10545.25
2,GAIL.csv,10000.0,10049.75
3,TCS.csv,10000.0,10585.65
4,TATASTEEL.csv,10000.0,10580.65
5,ICICIBANK.csv,10000.0,10249.80
6,HDFC.csv,10000.0,10628.20
7,BAJFINANCE.csv,10000.0,12529.40
8,GRASIM.csv,10000.0,10942.65
9,HDFCBANK.csv,10000.0,10187.95


In [ ]:
Summary_df

,stock_name,intial_portfolio_val,final_portfolio_val
0,HINDALCO.csv,10000.0,10207.35
1,RELIANCE.csv,10000.0,10545.25
2,GAIL.csv,10000.0,10049.75
3,TCS.csv,10000.0,10585.65
4,TATASTEEL.csv,10000.0,10580.65
5,ICICIBANK.csv,10000.0,10249.80
6,HDFC.csv,10000.0,10628.20
7,BAJFINANCE.csv,10000.0,12529.40
8,GRASIM.csv,10000.0,10942.65
9,HDFCBANK.csv,10000.0,10187.95


**Inverted Hammer Candle backtesting**

In [ ]:
!pip install mplfinance
!pip install backtrader
import glob
import pandas as pd
import backtrader as bt

In [ ]:
class Inverted_Hammer_candle_stick(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        
        self.dataclose = self.datas[0].close
        self.datahigh = self.datas[0].High
        self.dataopen = self.datas[0].Open
        self.datalow = self.datas[0].Low
        # To keep track of pending orders
        self.order = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

       
        if not self.position:
            if (((self.datahigh - self.datalow) > 3 * (self.dataopen - self.dataclose)) and((self.datahigh - self.dataclose) / (.001 + self.datahigh - self.datalow) > 0.6) and ((self.datahigh - self.dataopen) / (.001 + self.datahigh - self.datalow) > 0.6)):
                self.order = self.buy() 


        else:

            # Randomly selling after 3 timecyles...
            if len(self) >= (self.bar_executed + 3):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [ ]:
row=0
Summary_df=pd.DataFrame()
glued_data = pd.DataFrame()
for file_name in glob.glob('/content/Trading_algos/'+'*.csv'):
  if 'cerebro' in globals():
    del cerebro
  cerebro = bt.Cerebro()

  try:
    cerebro = bt.Cerebro()
    df= pd.read_csv(file_name, index_col='Date')
    df.index = pd.to_datetime(df.index) 
    df.drop(['Symbol','Series','Prev Close','Trades','Deliverable Volume','%Deliverble','Last','VWAP','Volume','Turnover'],inplace=True,axis=1)
    # For the period after specific date
    recent = df[df.index > '2020-05-29']
    # print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    intial_val=cerebro.broker.getvalue()
    
    data = bt.feeds.PandasData(dataname=recent)
    cerebro.adddata(data)
    cerebro.addstrategy(Inverted_Hammer_candle_stick)
    cerebro.run()
    final_val=cerebro.broker.getvalue()
    # print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    Summary_df.loc[row, 'stock_name'] = file_name.split("/")[-1]
    Summary_df.loc[row, 'intial_portfolio_val'] = intial_val
    Summary_df.loc[row, 'final_portfolio_val'] = final_val
    row+=1
  except Exception:
    pass

Streaming output truncated to the last 5000 lines.
2020-09-23, Close, 85.10
2020-09-24, Close, 82.65
2020-09-25, Close, 84.80
2020-09-28, Close, 87.50
2020-09-29, Close, 85.15
2020-09-30, Close, 85.10
2020-10-01, Close, 84.90
2020-10-05, Close, 84.45
2020-10-06, Close, 84.15
2020-10-07, Close, 82.80
2020-10-08, Close, 82.60
2020-10-09, Close, 82.90
2020-10-12, Close, 82.75
2020-10-13, Close, 82.65
2020-10-14, Close, 79.10
2020-10-15, Close, 79.00
2020-10-16, Close, 80.30
2020-10-19, Close, 81.65
2020-10-20, Close, 80.65
2020-10-21, Close, 82.30
2020-10-22, Close, 84.65
2020-10-23, Close, 86.25
2020-10-26, Close, 85.95
2020-10-27, Close, 89.45
2020-10-28, Close, 88.10
2020-10-29, Close, 86.50
2020-10-30, Close, 87.60
2020-11-02, Close, 89.20
2020-11-03, Close, 85.95
2020-11-04, Close, 85.10
2020-11-05, Close, 86.80
2020-11-06, Close, 87.30
2020-11-09, Close, 89.45
2020-11-10, Close, 91.50
2020-11-11, Close, 92.90
2020-11-12, Close, 90.80
2020-11-13, Close, 91.45
2020-11-14, Close, 90.90

In [ ]:
Summary_df

,stock_name,intial_portfolio_val,final_portfolio_val
0,HINDALCO.csv,10000.0,10000.0
1,RELIANCE.csv,10000.0,10000.0
2,GAIL.csv,10000.0,10000.0
3,TCS.csv,10000.0,10000.0
4,TATASTEEL.csv,10000.0,10000.0
5,ICICIBANK.csv,10000.0,10000.0
6,HDFC.csv,10000.0,10000.0
7,BAJFINANCE.csv,10000.0,10000.0
8,GRASIM.csv,10000.0,10000.0
9,HDFCBANK.csv,10000.0,10000.0


In [ ]:
class bull_dragon_fly(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        
        self.dataclose = self.datas[0].close
        self.datahigh = self.datas[0].High
        self.dataopen = self.datas[0].Open
        self.datalow = self.datas[0].Low
        # To keep track of pending orders
        self.order = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

       
        if not self.position:
            if (((self.datahigh - self.datalow) > 3 * (self.dataopen - self.dataclose)) and (self.dataclose - self.datalow) > 0.8* (self.datahigh - self.datalow) and (self.dataopen - self.datalow) > 0.8*(self.datahigh - self.datalow)):
                self.order = self.buy() 


        else:

            # Randomly selling after 3 timecyles...
            if len(self) >= (self.bar_executed + 3):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [ ]:
row=0
Summary_df=pd.DataFrame()
glued_data = pd.DataFrame()
for file_name in glob.glob('/content/Trading_algos/'+'*.csv'):
  if 'cerebro' in globals():
    del cerebro
  cerebro = bt.Cerebro()

  try:
    cerebro = bt.Cerebro()
    df= pd.read_csv(file_name, index_col='Date')
    df.index = pd.to_datetime(df.index) 
    df.drop(['Symbol','Series','Prev Close','Trades','Deliverable Volume','%Deliverble','Last','VWAP','Volume','Turnover'],inplace=True,axis=1)
    # For the period after specific date
    recent = df[df.index > '2020-05-31']
    # print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    intial_val=cerebro.broker.getvalue()
    
    data = bt.feeds.PandasData(dataname=recent)
    cerebro.adddata(data)
    cerebro.addstrategy(bull_dragon_fly)
    cerebro.run()
    final_val=cerebro.broker.getvalue()
    # print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    Summary_df.loc[row, 'stock_name'] = file_name.split("/")[-1]
    Summary_df.loc[row, 'intial_portfolio_val'] = intial_val
    Summary_df.loc[row, 'final_portfolio_val'] = final_val
    row+=1
  except Exception:
    pass

Streaming output truncated to the last 5000 lines.
2020-11-14, SELL CREATE, 514.00
2020-11-17, SELL EXECUTED, 517.00
2020-11-17, Close, 518.40
2020-11-18, BUY EXECUTED, 514.00
2020-11-18, Close, 513.00
2020-11-19, Close, 507.35
2020-11-20, Close, 503.25
2020-11-23, Close, 512.85
2020-11-23, SELL CREATE, 512.85
2020-11-24, SELL EXECUTED, 513.50
2020-11-24, Close, 520.15
2020-11-25, BUY EXECUTED, 521.20
2020-11-25, Close, 506.20
2020-11-26, Close, 511.70
2020-11-27, Close, 511.65
2020-12-01, Close, 539.35
2020-12-01, SELL CREATE, 539.35
2020-12-02, SELL EXECUTED, 540.90
2020-12-02, Close, 544.65
2020-12-03, BUY EXECUTED, 550.00
2020-12-03, Close, 548.25
2020-12-04, Close, 569.05
2020-12-07, Close, 582.30
2020-12-08, Close, 568.70
2020-12-08, SELL CREATE, 568.70
2020-12-09, SELL EXECUTED, 571.00
2020-12-09, Close, 569.55
2020-12-10, BUY EXECUTED, 570.00
2020-12-10, Close, 565.20
2020-12-11, Close, 568.10
2020-12-14, Close, 576.25
2020-12-15, Close, 571.65
2020-12-15, SELL CREATE, 571.65
2

In [ ]:
Summary_df

,stock_name,intial_portfolio_val,final_portfolio_val
0,HINDALCO.csv,10000.0,10207.35
1,RELIANCE.csv,10000.0,10545.25
2,GAIL.csv,10000.0,10049.75
3,TCS.csv,10000.0,10585.65
4,TATASTEEL.csv,10000.0,10580.65
5,ICICIBANK.csv,10000.0,10249.80
6,HDFC.csv,10000.0,10628.20
7,BAJFINANCE.csv,10000.0,12529.40
8,GRASIM.csv,10000.0,10942.65
9,HDFCBANK.csv,10000.0,10187.95


In [ ]:
Summary_all=Summary_df.sum()
print("Profit: "+str(Summary_all['final_portfolio_val'] - Summary_all['intial_portfolio_val']) +" for the period: "+str(recent.reset_index().loc[0,'Date'])+ " to "+str(recent.reset_index().loc[len(recent)-1,'Date']))


Profit: 25769.0 for the period: 2020-06-01 00:00:00 to 2021-04-30 00:00:00


In [ ]:
class black_spinning_top(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        
        self.dataclose = self.datas[0].close
        self.datahigh = self.datas[0].High
        self.dataopen = self.datas[0].Open
        self.datalow = self.datas[0].Low
        # To keep track of pending orders
        self.order = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

       
        if not self.position:
            if (((self.dataopen > self.dataclose) and (self.datahigh - self.datalow) > 3*(self.dataopen - self.dataclose) and (((self.datahigh - self.dataopen)/(0.001+self.datahigh - self.datalow))<0.4) and (((self.dataclose-self.datalow)/(0.001+self.datahigh-self.datalow))<0.4))):
                self.order = self.buy() 


        else:

            # Randomly selling after 3 timecyles...
            if len(self) >= (self.bar_executed + 3):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [ ]:
row=0
Summary_df=pd.DataFrame()
glued_data = pd.DataFrame()
for file_name in glob.glob('/content/Trading_algos/'+'*.csv'):
  if 'cerebro' in globals():
    del cerebro
  cerebro = bt.Cerebro()

  try:
    cerebro = bt.Cerebro()
    df= pd.read_csv(file_name, index_col='Date')
    df.index = pd.to_datetime(df.index) 
    df.drop(['Symbol','Series','Prev Close','Trades','Deliverable Volume','%Deliverble','Last','VWAP','Volume','Turnover'],inplace=True,axis=1)
    # For the period after specific date
    recent = df[df.index > '2020-05-31']
    # print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    intial_val=cerebro.broker.getvalue()
    
    data = bt.feeds.PandasData(dataname=recent)
    cerebro.adddata(data)
    cerebro.addstrategy(black_spinning_top)
    cerebro.run()
    final_val=cerebro.broker.getvalue()
    # print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    Summary_df.loc[row, 'stock_name'] = file_name.split("/")[-1]
    Summary_df.loc[row, 'intial_portfolio_val'] = intial_val
    Summary_df.loc[row, 'final_portfolio_val'] = final_val
    row+=1
  except Exception:
    pass

Streaming output truncated to the last 5000 lines.
2020-09-23, Close, 85.10
2020-09-24, Close, 82.65
2020-09-25, Close, 84.80
2020-09-28, Close, 87.50
2020-09-29, Close, 85.15
2020-09-30, Close, 85.10
2020-10-01, Close, 84.90
2020-10-05, Close, 84.45
2020-10-06, Close, 84.15
2020-10-07, Close, 82.80
2020-10-08, Close, 82.60
2020-10-09, Close, 82.90
2020-10-12, Close, 82.75
2020-10-13, Close, 82.65
2020-10-14, Close, 79.10
2020-10-15, Close, 79.00
2020-10-16, Close, 80.30
2020-10-19, Close, 81.65
2020-10-20, Close, 80.65
2020-10-21, Close, 82.30
2020-10-22, Close, 84.65
2020-10-23, Close, 86.25
2020-10-26, Close, 85.95
2020-10-27, Close, 89.45
2020-10-28, Close, 88.10
2020-10-29, Close, 86.50
2020-10-30, Close, 87.60
2020-11-02, Close, 89.20
2020-11-03, Close, 85.95
2020-11-04, Close, 85.10
2020-11-05, Close, 86.80
2020-11-06, Close, 87.30
2020-11-09, Close, 89.45
2020-11-10, Close, 91.50
2020-11-11, Close, 92.90
2020-11-12, Close, 90.80
2020-11-13, Close, 91.45
2020-11-14, Close, 90.90

In [ ]:
Summary_df

,stock_name,intial_portfolio_val,final_portfolio_val
0,HINDALCO.csv,10000.0,10000.0
1,RELIANCE.csv,10000.0,10000.0
2,GAIL.csv,10000.0,10000.0
3,TCS.csv,10000.0,10000.0
4,TATASTEEL.csv,10000.0,10000.0
5,ICICIBANK.csv,10000.0,10000.0
6,HDFC.csv,10000.0,10000.0
7,BAJFINANCE.csv,10000.0,10000.0
8,GRASIM.csv,10000.0,10000.0
9,HDFCBANK.csv,10000.0,10000.0


In [ ]:
class white_spinning_top(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        
        self.dataclose = self.datas[0].close
        self.datahigh = self.datas[0].High
        self.dataopen = self.datas[0].Open
        self.datalow = self.datas[0].Low
        # To keep track of pending orders
        self.order = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

       
        if not self.position:
            if (self.dataclose > self.dataopen) and ((self.datahigh - self.datalow) > 3*(self.dataclose - self.dataopen)) and (((self.datahigh - self.dataclose)/(0.001+self.datahigh - self.datalow))<0.4) and (((self.dataopen-self.datalow)/(0.001+self.datahigh-self.datalow))<0.4):
                self.order = self.buy() 


        else:

            # Randomly selling after 3 timecyles...
            if len(self) >= (self.bar_executed + 3):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [ ]:
row=0
Summary_df=pd.DataFrame()
glued_data = pd.DataFrame()
for file_name in glob.glob('/content/Trading_algos/'+'*.csv'):
  if 'cerebro' in globals():
    del cerebro
  cerebro = bt.Cerebro()

  try:
    cerebro = bt.Cerebro()
    df= pd.read_csv(file_name, index_col='Date')
    df.index = pd.to_datetime(df.index) 
    df.drop(['Symbol','Series','Prev Close','Trades','Deliverable Volume','%Deliverble','Last','VWAP','Volume','Turnover'],inplace=True,axis=1)
    # For the period after specific date
    recent = df[df.index > '2020-05-31']
    # print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    intial_val=cerebro.broker.getvalue()
    
    data = bt.feeds.PandasData(dataname=recent)
    cerebro.adddata(data)
    cerebro.addstrategy(white_spinning_top)
    cerebro.run()
    final_val=cerebro.broker.getvalue()
    # print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    Summary_df.loc[row, 'stock_name'] = file_name.split("/")[-1]
    Summary_df.loc[row, 'intial_portfolio_val'] = intial_val
    Summary_df.loc[row, 'final_portfolio_val'] = final_val
    row+=1
  except Exception:
    pass

Streaming output truncated to the last 5000 lines.
2020-09-23, Close, 85.10
2020-09-24, Close, 82.65
2020-09-25, Close, 84.80
2020-09-28, Close, 87.50
2020-09-29, Close, 85.15
2020-09-30, Close, 85.10
2020-10-01, Close, 84.90
2020-10-05, Close, 84.45
2020-10-06, Close, 84.15
2020-10-07, Close, 82.80
2020-10-08, Close, 82.60
2020-10-09, Close, 82.90
2020-10-12, Close, 82.75
2020-10-13, Close, 82.65
2020-10-14, Close, 79.10
2020-10-15, Close, 79.00
2020-10-16, Close, 80.30
2020-10-19, Close, 81.65
2020-10-20, Close, 80.65
2020-10-21, Close, 82.30
2020-10-22, Close, 84.65
2020-10-23, Close, 86.25
2020-10-26, Close, 85.95
2020-10-27, Close, 89.45
2020-10-28, Close, 88.10
2020-10-29, Close, 86.50
2020-10-30, Close, 87.60
2020-11-02, Close, 89.20
2020-11-03, Close, 85.95
2020-11-04, Close, 85.10
2020-11-05, Close, 86.80
2020-11-06, Close, 87.30
2020-11-09, Close, 89.45
2020-11-10, Close, 91.50
2020-11-11, Close, 92.90
2020-11-12, Close, 90.80
2020-11-13, Close, 91.45
2020-11-14, Close, 90.90

In [ ]:
Summary_df

,stock_name,intial_portfolio_val,final_portfolio_val
0,HINDALCO.csv,10000.0,10000.0
1,RELIANCE.csv,10000.0,10000.0
2,GAIL.csv,10000.0,10000.0
3,TCS.csv,10000.0,10000.0
4,TATASTEEL.csv,10000.0,10000.0
5,ICICIBANK.csv,10000.0,10000.0
6,HDFC.csv,10000.0,10000.0
7,BAJFINANCE.csv,10000.0,10000.0
8,GRASIM.csv,10000.0,10000.0
9,HDFCBANK.csv,10000.0,10000.0


In [ ]:
class hanging_man(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        
        self.dataclose = self.datas[0].close
        self.datahigh = self.datas[0].High
        self.dataopen = self.datas[0].Open
        self.datalow = self.datas[0].Low
        # To keep track of pending orders
        self.order = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

       
        if not self.position:
            if (self.datahigh - self.datalow) > 4*(self.dataopen-self.dataclose) and ((self.dataclose - self.datalow)/(0.001+self.datahigh - self.datalow)) >= 0.75 and (((self.dataopen - self.datalow)/(0.001+self.datahigh - self.datalow)) >= 0.75):
                self.order = self.buy() 


        else:

            # Randomly selling after 3 timecyles...
            if len(self) >= (self.bar_executed + 3):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [ ]:
row=0
Summary_df=pd.DataFrame()
glued_data = pd.DataFrame()
for file_name in glob.glob('/content/Trading_algos/'+'*.csv'):
  if 'cerebro' in globals():
    del cerebro
  cerebro = bt.Cerebro()

  try:
    cerebro = bt.Cerebro()
    df= pd.read_csv(file_name, index_col='Date')
    df.index = pd.to_datetime(df.index) 
    df.drop(['Symbol','Series','Prev Close','Trades','Deliverable Volume','%Deliverble','Last','VWAP','Volume','Turnover'],inplace=True,axis=1)
    # For the period after specific date
    recent = df[df.index > '2020-05-31']
    # print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    intial_val=cerebro.broker.getvalue()
    
    data = bt.feeds.PandasData(dataname=recent)
    cerebro.adddata(data)
    cerebro.addstrategy(hanging_man)
    cerebro.run()
    final_val=cerebro.broker.getvalue()
    # print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    Summary_df.loc[row, 'stock_name'] = file_name.split("/")[-1]
    Summary_df.loc[row, 'intial_portfolio_val'] = intial_val
    Summary_df.loc[row, 'final_portfolio_val'] = final_val
    row+=1
  except Exception:
    pass

Streaming output truncated to the last 5000 lines.
2020-11-14, SELL CREATE, 514.00
2020-11-17, SELL EXECUTED, 517.00
2020-11-17, Close, 518.40
2020-11-18, BUY EXECUTED, 514.00
2020-11-18, Close, 513.00
2020-11-19, Close, 507.35
2020-11-20, Close, 503.25
2020-11-23, Close, 512.85
2020-11-23, SELL CREATE, 512.85
2020-11-24, SELL EXECUTED, 513.50
2020-11-24, Close, 520.15
2020-11-25, BUY EXECUTED, 521.20
2020-11-25, Close, 506.20
2020-11-26, Close, 511.70
2020-11-27, Close, 511.65
2020-12-01, Close, 539.35
2020-12-01, SELL CREATE, 539.35
2020-12-02, SELL EXECUTED, 540.90
2020-12-02, Close, 544.65
2020-12-03, BUY EXECUTED, 550.00
2020-12-03, Close, 548.25
2020-12-04, Close, 569.05
2020-12-07, Close, 582.30
2020-12-08, Close, 568.70
2020-12-08, SELL CREATE, 568.70
2020-12-09, SELL EXECUTED, 571.00
2020-12-09, Close, 569.55
2020-12-10, BUY EXECUTED, 570.00
2020-12-10, Close, 565.20
2020-12-11, Close, 568.10
2020-12-14, Close, 576.25
2020-12-15, Close, 571.65
2020-12-15, SELL CREATE, 571.65
2

In [ ]:
Summary_df

,stock_name,intial_portfolio_val,final_portfolio_val
0,HINDALCO.csv,10000.0,10207.35
1,RELIANCE.csv,10000.0,10545.25
2,GAIL.csv,10000.0,10049.75
3,TCS.csv,10000.0,10585.65
4,TATASTEEL.csv,10000.0,10580.65
5,ICICIBANK.csv,10000.0,10249.80
6,HDFC.csv,10000.0,10628.20
7,BAJFINANCE.csv,10000.0,12529.40
8,GRASIM.csv,10000.0,10942.65
9,HDFCBANK.csv,10000.0,10187.95


In [ ]:
from datetime import datetime
import backtrader as bt

class SmaCross(bt.SignalStrategy):
    def __init__(self):
        sma1, sma2 = bt.ind.SMA(period=9), bt.ind.SMA(period=30)
        crossover = bt.ind.CrossOver(sma1, sma2)
        self.signal_add(bt.SIGNAL_LONG, crossover)

In [ ]:
row=0
Summary_df=pd.DataFrame()
glued_data = pd.DataFrame()
for file_name in glob.glob('/content/Trading_algos/'+'*.csv'):
  if 'cerebro' in globals():
    del cerebro
  cerebro = bt.Cerebro()

  try:
    cerebro = bt.Cerebro()
    df= pd.read_csv(file_name, index_col='Date')
    df.index = pd.to_datetime(df.index) 
    df.drop(['Symbol','Series','Prev Close','Trades','Deliverable Volume','%Deliverble','Last','VWAP','Volume','Turnover'],inplace=True,axis=1)
    # For the period after specific date
    recent = df[df.index > '2020-05-31']
    # print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    intial_val=cerebro.broker.getvalue()
    
    data = bt.feeds.PandasData(dataname=recent)
    cerebro.adddata(data)
    cerebro.addstrategy(SmaCross)
    cerebro.run()
    final_val=cerebro.broker.getvalue()
    # print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    Summary_df.loc[row, 'stock_name'] = file_name.split("/")[-1]
    Summary_df.loc[row, 'intial_portfolio_val'] = intial_val
    Summary_df.loc[row, 'final_portfolio_val'] = final_val
    row+=1
  except Exception:
    pass

In [ ]:
Summary_df

,stock_name,intial_portfolio_val,final_portfolio_val
0,HINDALCO.csv,10000.0,10061.55
1,RELIANCE.csv,10000.0,9618.80
2,GAIL.csv,10000.0,10018.50
3,TCS.csv,10000.0,10759.10
4,TATASTEEL.csv,10000.0,10549.90
5,ICICIBANK.csv,10000.0,10208.60
6,HDFC.csv,10000.0,9974.90
7,BAJFINANCE.csv,10000.0,11570.05
8,GRASIM.csv,10000.0,10635.90
9,HDFCBANK.csv,10000.0,10274.65


In [ ]:
class MAcrossover(bt.Strategy): 
    # Moving average parameters
    params = (('pfast',20),('pslow',5),)

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()} {txt}') # Comment this line when running optimization

    def __init__(self):
        self.dataclose = self.datas[0].close
        
		# Order variable will contain ongoing order details/status
        self.order = None

        # Instantiate moving averages
        self.slow_sma = bt.indicators.MovingAverageSimple(self.datas[0], 
                        period=self.params.pslow)
        self.fast_sma = bt.indicators.MovingAverageSimple(self.datas[0], 
                        period=self.params.pfast)

In [ ]:
row=0
Summary_df=pd.DataFrame()
glued_data = pd.DataFrame()
for file_name in glob.glob('/content/Trading_algos/'+'*.csv'):
  if 'cerebro' in globals():
    del cerebro
  cerebro = bt.Cerebro()

  try:
    cerebro = bt.Cerebro()
    df= pd.read_csv(file_name, index_col='Date')
    df.index = pd.to_datetime(df.index) 
    df.drop(['Symbol','Series','Prev Close','Trades','Deliverable Volume','%Deliverble','Last','VWAP','Volume','Turnover'],inplace=True,axis=1)
    # For the period after specific date
    recent = df[df.index > '2020-05-31']
    # print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    intial_val=cerebro.broker.getvalue()
    
    data = bt.feeds.PandasData(dataname=recent)
    cerebro.adddata(data)
    cerebro.addstrategy(MAcrossover)
    cerebro.run()
    final_val=cerebro.broker.getvalue()
    # print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    Summary_df.loc[row, 'stock_name'] = file_name.split("/")[-1]
    Summary_df.loc[row, 'intial_portfolio_val'] = intial_val
    Summary_df.loc[row, 'final_portfolio_val'] = final_val
    row+=1
  except Exception:
    pass

In [ ]:
Summary_df

,stock_name,intial_portfolio_val,final_portfolio_val
0,HINDALCO.csv,10000.0,10000.0
1,RELIANCE.csv,10000.0,10000.0
2,GAIL.csv,10000.0,10000.0
3,TCS.csv,10000.0,10000.0
4,TATASTEEL.csv,10000.0,10000.0
5,ICICIBANK.csv,10000.0,10000.0
6,HDFC.csv,10000.0,10000.0
7,BAJFINANCE.csv,10000.0,10000.0
8,GRASIM.csv,10000.0,10000.0
9,HDFCBANK.csv,10000.0,10000.0


In [ ]:
class BtcSentiment(bt.Strategy):

  params = (('period', 10), ('devfactor', 1),)

  def log(self, txt, dt=None):
      dt = dt or self.datas[0].datetime.date(0)
      print(f'{dt.isoformat()} {txt}')

  def __init__(self):
      self.btc_price = self.datas[0].close
      self.google_sentiment = self.datas[1].close
      self.bbands = bt.indicators.BollingerBands(self.google_sentiment,
              period=self.params.period, devfactor=self.params.devfactor)

      self.order = None

  def notify_order(self, order):
      if order.status in [order.Submitted, order.Accepted]:
          # Existing order - Nothing to do
          return

      # Check if an order has been completed
      # Attention: broker could reject order if not enough cash


      if order.status in [order.Completed]:

        if order.isbuy():


          self.log(f'BUY EXECUTED, {order.executed.price:.2f}')
        elif order.issell():

          self.log(f'SELL EXECUTED, {order.executed.price:.2f}')
          self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin,order.Rejected]:
                              
          self.log('Order Canceled/Margin/Rejected')

          # Reset orders
          self.order = None

      def next(self):
          # Check for open orders
          if self.order:
              return

      #Long signal 
      if self.google_sentiment > self.bbands.lines.top[0]:
        # Check if we are in the market
        if not self.position:
          self.log(f'Google Sentiment Value: {self.google_sentiment[0]:.2f}')
          self.log(f'Top band: {self.bbands.lines.top[0]:.2f}')
          # We are not in the market, we will open a trade
          self.log(f'***BUY CREATE {self.btc_price[0]:.2f}')
          # Keep track of the created order to avoid a 2nd order
          self.order = self.buy()       

      #Short signal
      elif self.google_sentiment < self.bbands.lines.bot[0]:

        # Check if we are in the market
        if not self.position:
          self.log(f'Google Sentiment Value: {self.google_sentiment[0]:.2f}')
          self.log(f'Bottom band: {self.bbands.lines.bot[0]:.2f}')
          # We are not in the market, we will open a trade
          self.log(f'***SELL CREATE {self.btc_price[0]:.2f}')
          # Keep track of the created order to avoid a 2nd order
          self.order = self.sell()
      
      #Neutral signal - close any open trades     
      else:
        if self.position:
          # We are in the market, we will close the existing trade
          self.log(f'Google Sentiment Value: {self.google_sentiment[0]:.2f}')
          self.log(f'Bottom band: {self.bbands.lines.bot[0]:.2f}')
          self.log(f'Top band: {self.bbands.lines.top[0]:.2f}')
          self.log(f'CLOSE CREATE {self.btc_price[0]:.2f}')
          self.order = self.close()

In [ ]:
row=0
Summary_df=pd.DataFrame()
glued_data = pd.DataFrame()
for file_name in glob.glob('/content/Trading_algos/'+'*.csv'):
  if 'cerebro' in globals():
    del cerebro
  cerebro = bt.Cerebro()

  try:
    cerebro = bt.Cerebro()
    df= pd.read_csv(file_name, index_col='Date')
    df.index = pd.to_datetime(df.index) 
    df.drop(['Symbol','Series','Prev Close','Trades','Deliverable Volume','%Deliverble','Last','VWAP','Volume','Turnover'],inplace=True,axis=1)
    # For the period after specific date
    recent = df[df.index > '2020-05-31']
    # print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    intial_val=cerebro.broker.getvalue()
    
    data = bt.feeds.PandasData(dataname=recent)
    cerebro.adddata(data)
    cerebro.addstrategy(MAcrossover)
    cerebro.run()
    final_val=cerebro.broker.getvalue()
    # print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    Summary_df.loc[row, 'stock_name'] = file_name.split("/")[-1]
    Summary_df.loc[row, 'intial_portfolio_val'] = intial_val
    Summary_df.loc[row, 'final_portfolio_val'] = final_val
    row+=1
  except Exception:
    pass

In [ ]:
Summary_df

,stock_name,intial_portfolio_val,final_portfolio_val
0,HINDALCO.csv,10000.0,10000.0
1,RELIANCE.csv,10000.0,10000.0
2,GAIL.csv,10000.0,10000.0
3,TCS.csv,10000.0,10000.0
4,TATASTEEL.csv,10000.0,10000.0
5,ICICIBANK.csv,10000.0,10000.0
6,HDFC.csv,10000.0,10000.0
7,BAJFINANCE.csv,10000.0,10000.0
8,GRASIM.csv,10000.0,10000.0
9,HDFCBANK.csv,10000.0,10000.0


In [ ]:
class AverageTrueRange(bt.Strategy):

	def log(self, txt, dt=None):
		dt = dt or self.datas[0].datetime.date(0)
		print(f'{dt.isoformat()} {txt}') #Print date and close
		
	def __init__(self):
		self.dataclose = self.datas[0].close
		self.datahigh = self.datas[0].high
		self.datalow = self.datas[0].low
		
	def next(self):
		range_total = 0
		for i in range(-13, 1):
			true_range = self.datahigh[i] - self.datalow[i]
			range_total += true_range
		ATR = range_total / 14

		self.log(f'Close: {self.dataclose[0]:.2f}, ATR: {ATR:.4f}')

In [ ]:
row=0
Summary_df=pd.DataFrame()
glued_data = pd.DataFrame()
for file_name in glob.glob('/content/Trading_algos/'+'*.csv'):
  if 'cerebro' in globals():
    del cerebro
  cerebro = bt.Cerebro()

  try:
    cerebro = bt.Cerebro()
    df= pd.read_csv(file_name, index_col='Date')
    df.index = pd.to_datetime(df.index) 
    df.drop(['Symbol','Series','Prev Close','Trades','Deliverable Volume','%Deliverble','Last','VWAP','Volume','Turnover'],inplace=True,axis=1)
    # For the period after specific date
    recent = df[df.index > '2020-05-31']
    # print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    intial_val=cerebro.broker.getvalue()
    
    data = bt.feeds.PandasData(dataname=recent)
    cerebro.adddata(data)
    cerebro.addstrategy(MAcrossover)
    cerebro.run()
    final_val=cerebro.broker.getvalue()
    # print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    Summary_df.loc[row, 'stock_name'] = file_name.split("/")[-1]
    Summary_df.loc[row, 'intial_portfolio_val'] = intial_val
    Summary_df.loc[row, 'final_portfolio_val'] = final_val
    row+=1
  except Exception:
    pass

In [ ]:
Summary_df

,stock_name,intial_portfolio_val,final_portfolio_val
0,HINDALCO.csv,10000.0,10000.0
1,RELIANCE.csv,10000.0,10000.0
2,GAIL.csv,10000.0,10000.0
3,TCS.csv,10000.0,10000.0
4,TATASTEEL.csv,10000.0,10000.0
5,ICICIBANK.csv,10000.0,10000.0
6,HDFC.csv,10000.0,10000.0
7,BAJFINANCE.csv,10000.0,10000.0
8,GRASIM.csv,10000.0,10000.0
9,HDFCBANK.csv,10000.0,10000.0
